In [217]:
import dask.dataframe as dd
import pandas as pd
import feather as ft

In [220]:
# df_full = dd.read_csv("data/weeplace_checkins.csv")
# # df_full = df_full.dropna(subset=["placeid"])
# df_full = pd.read_csv("data/weeplace_checkins.csv")
# ft.write_dataframe(df_full, 'data/weeplace_checkins.feather')
df_full = ft.read_dataframe('data/weeplace_checkins.feather')

In [221]:
df_full.head(10)

,userid,placeid,datetime,lat,lon,city,category
0,fred-wilson,tocqueville-new-york,2010-10-22 23:00:00,40.736300,-73.992200,New York,Food:American
1,fred-wilson,wesleyan-university-usdan-university-center-mi...,2010-10-23 17:00:00,41.556974,-72.657571,Middletown,College & Education:Student Center
2,fred-wilson,javapalooza-middletown,2010-10-23 20:00:00,41.558853,-72.648618,Middletown,Food:Coffee Shop
3,fred-wilson,giorgione-new-york,2010-10-24 00:00:00,40.726144,-74.008348,New York,Food:Italian
4,fred-wilson,the-shala-new-york,2010-10-24 16:00:00,40.733075,-73.991478,New York,Home / Work / Other:Gym / Fitness:Yoga Studio
5,fred-wilson,citarella-new-york,2010-10-24 19:00:00,40.734400,-73.999000,New York,Shops:Food & Drink:Gourmet
6,fred-wilson,jeffreys-grocery-and-luncheonette-new-york,2010-10-25 12:00:00,40.733994,-74.001374,New York,Food:American
7,fred-wilson,jacks-coffee-new-york,2010-10-25 12:00:00,40.734578,-74.000636,New York,Food:Coffee Shop
8,fred-wilson,chelsea-piers-sports-center-new-york,2010-10-26 10:00:00,40.746900,-74.008200,New York,Parks & Outdoors:Skating Rink
9,fred-wilson,hudson-river-park-new-york,2010-10-26 11:00:00,40.731682,-74.010773,New York,Parks & Outdoors:Other - Parks & Outdoor


In [30]:
# make new dataframe removing minutes and seconds information, called df_hour
# df_full['datetime'] = dd.to_datetime(df_full['datetime']).dt.floor('H')
df_full['datetime'] = pd.to_datetime(df_full['datetime']).dt.floor('H')
df_hour = df_full

In [31]:
df_hour

,userid,placeid,datetime,lat,lon,city,category
0,fred-wilson,tocqueville-new-york,2010-10-22 23:00:00,40.736300,-73.992200,New York,Food:American
1,fred-wilson,wesleyan-university-usdan-university-center-mi...,2010-10-23 17:00:00,41.556974,-72.657571,Middletown,College & Education:Student Center
2,fred-wilson,javapalooza-middletown,2010-10-23 20:00:00,41.558853,-72.648618,Middletown,Food:Coffee Shop
3,fred-wilson,giorgione-new-york,2010-10-24 00:00:00,40.726144,-74.008348,New York,Food:Italian
4,fred-wilson,the-shala-new-york,2010-10-24 16:00:00,40.733075,-73.991478,New York,Home / Work / Other:Gym / Fitness:Yoga Studio
5,fred-wilson,citarella-new-york,2010-10-24 19:00:00,40.734400,-73.999000,New York,Shops:Food & Drink:Gourmet
6,fred-wilson,jeffreys-grocery-and-luncheonette-new-york,2010-10-25 12:00:00,40.733994,-74.001374,New York,Food:American
7,fred-wilson,jacks-coffee-new-york,2010-10-25 12:00:00,40.734578,-74.000636,New York,Food:Coffee Shop
8,fred-wilson,chelsea-piers-sports-center-new-york,2010-10-26 10:00:00,40.746900,-74.008200,New York,Parks & Outdoors:Skating Rink
9,fred-wilson,hudson-river-park-new-york,2010-10-26 11:00:00,40.731682,-74.010773,New York,Parks & Outdoors:Other - Parks & Outdoor


In [216]:
# for each userid, get the range of datetime
df_time_range = df_hour.groupby('userid').agg({'datetime': ['max', 'min']}).datetime
# create full hourly time (empty) slots in the range for each userid
df_full_time = df_time_range.apply(lambda x: pd.date_range(x['min'], x['max'], freq = '1H'), axis = 1)
df_full_time

userid
00-a                 DatetimeIndex(['2010-02-02 13:00:00', '2010-02...
0046aki              DatetimeIndex(['2010-04-29 06:00:00', '2010-04...
01                   DatetimeIndex(['2010-05-07 18:00:00', '2010-05...
0403                 DatetimeIndex(['2010-11-21 00:00:00', '2010-11...
062                  DatetimeIndex(['2011-04-03 14:00:00', '2011-04...
08                   DatetimeIndex(['2009-11-24 11:00:00', '2009-11...
1-2-punch            DatetimeIndex(['2010-04-18 23:00:00', '2010-04...
12                   DatetimeIndex(['2010-01-20 09:00:00', '2010-01...
13                   DatetimeIndex(['2011-02-15 12:00:00', '2011-02...
172                  DatetimeIndex(['2011-01-24 13:00:00', '2011-01...
1od                  DatetimeIndex(['2009-10-19 15:00:00', '2009-10...
26                   DatetimeIndex(['2010-01-22 06:00:00', '2010-01...
29                   DatetimeIndex(['2010-05-12 08:00:00', '2010-05...
3                    DatetimeIndex(['2010-05-31 11:00:00', '2010-05...

In [222]:
# create new dataframe with userid and full datatime-hourly
# l1 = []
# l2 = []
# for idx in df_full_time .index:
#     for itm in df_full_time [idx]:
#         l1.append(idx)
#         l2.append(itm)

# df_temp = pd.DataFrame()
# df_temp['userid'] = l1
# df_temp['datetime'] = l2

df_temp = ft.read_dataframe('data/weeplace_checkins_hourly_full_time.feather')

In [224]:
# ft.write_dataframe(df_temp, 'data/weeplace_checkins_hourly_full_time.feather')

In [225]:
# df_hourly_datetime = df_temp.append(df_hour, ignore_index=True, sort=False).drop_duplicates(['userid','datetime'], keep='last')
df_hourly_datetime = ft.read_dataframe('data/weeplace_checkins_hourly_datetime.feather')

In [227]:
# ft.write_dataframe(df_hourly_datetime, 'data/weeplace_checkins_hourly_datetime.feather')

Index(['00-a', '0046aki', '01'], dtype='object', name='userid')

DatetimeIndex(['2010-02-02 13:00:00', '2010-02-02 14:00:00',
               '2010-02-02 15:00:00', '2010-02-02 16:00:00',
               '2010-02-02 17:00:00', '2010-02-02 18:00:00',
               '2010-02-02 19:00:00', '2010-02-02 20:00:00',
               '2010-02-02 21:00:00', '2010-02-02 22:00:00',
               ...
               '2010-10-21 03:00:00', '2010-10-21 04:00:00',
               '2010-10-21 05:00:00', '2010-10-21 06:00:00',
               '2010-10-21 07:00:00', '2010-10-21 08:00:00',
               '2010-10-21 09:00:00', '2010-10-21 10:00:00',
               '2010-10-21 11:00:00', '2010-10-21 12:00:00'],
              dtype='datetime64[ns]', length=6264, freq='H')

In [212]:
k.sort_values(by='datetime').head(5)

,userid,datetime,placeid,lat,lon,city,category
20591,00-a,2010-02-02 13:00:00,shibuya-station,35.658656,139.701161,渋谷区,Travel:Train Station:Train
1,00-a,2010-02-02 14:00:00,NaN,NaN,NaN,NaN,NaN
2,00-a,2010-02-02 15:00:00,NaN,NaN,NaN,NaN,NaN
3,00-a,2010-02-02 16:00:00,NaN,NaN,NaN,NaN,NaN
4,00-a,2010-02-02 17:00:00,NaN,NaN,NaN,NaN,NaN


In [213]:
k.sort_values(by='datetime').tail(5)

,userid,datetime,placeid,lat,lon,city,category
19816,01,2011-04-22 09:00:00,NaN,NaN,NaN,NaN,NaN
19817,01,2011-04-22 10:00:00,NaN,NaN,NaN,NaN,NaN
19818,01,2011-04-22 11:00:00,NaN,NaN,NaN,NaN,NaN
19819,01,2011-04-22 12:00:00,NaN,NaN,NaN,NaN,NaN
19820,01,2011-04-22 13:00:00,NaN,NaN,NaN,NaN,NaN


In [214]:
m = k.loc[k['userid'] == '00-a']
m.sort_values(by='datetime').tail(5)

,userid,datetime,placeid,lat,lon,city,category
20091,00-a,2010-10-21 08:00:00,NaN,35.650117,139.720583,渋谷区,Food:Sandwiches
6260,00-a,2010-10-21 09:00:00,NaN,NaN,NaN,NaN,NaN
6261,00-a,2010-10-21 10:00:00,NaN,NaN,NaN,NaN,NaN
20092,00-a,2010-10-21 11:00:00,NaN,35.648710,139.715858,渋谷区,Shops:Other - Shopping
20093,00-a,2010-10-21 12:00:00,806540058,35.650241,139.718498,NaN,NaN
